In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from citipy import citipy
from datetime import datetime
import time
import requests
from config import weather_api_keys


In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-87.76184903668826 178.76690965974308
-30.781119168586933 -165.36320565110202
54.01855505778076 25.60116971164652
67.18600142394845 -75.56384866705893
-70.40269851596929 114.30052550416474
72.63062792987398 -92.71919645635793
42.964506391160825 148.51351946217568
57.05321345922755 95.11775080563496
-71.60612130448807 -115.6357464732104
-76.0296768395724 121.41007943332437
59.22402675110837 -137.0848749474152
28.562277458874746 -89.35649486451655
-35.00399821497911 15.187151940620112
-84.35975556204761 -132.04198216365117
9.524200923424587 -161.10870175236437
-10.772055484766241 116.65335477889431
-34.695995219680185 82.62907188701809
-88.50471548964852 121.79196867868592
36.59669803843222 66.13458962540906
71.4021031347946 51.82471357844173
-61.65283172117583 41.875701422978324
62.466436910692494 48.09417103101208
28.1497508841492 -33.07166500742352
9.041412834972704 -77.69034609033037
-33.57770691751179 4.613466064422568
-10.258389397499045 -145.4833201310548
-79.9285119669237 162.811

In [4]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)


792

### Most of this is copied straight from WeatherPy.ipynb

In [7]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_keys

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
#lets not do this again.....

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]   
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Current Description": city_weather_description,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | iqaluit
Processing Record 2 of Set 1 | moose factory
Processing Record 3 of Set 1 | rikitea
Processing Record 4 of Set 1 | beyneu
Processing Record 5 of Set 1 | puerto ayora
Processing Record 6 of Set 1 | arraial do cabo
Processing Record 7 of Set 1 | dassel
Processing Record 8 of Set 1 | huarmey
Processing Record 9 of Set 1 | new guinlo
City not found. Skipping...
Processing Record 10 of Set 1 | kavieng
Processing Record 11 of Set 1 | sagar
Processing Record 12 of Set 1 | hobart
Processing Record 13 of Set 1 | albany
Processing Record 14 of Set 1 | port alfred
Processing Record 15 of Set 1 | mataura
Processing Record 16 of Set 1 | kapaa
Processing Record 17 of Set 1 | maunabo
Processing Record 18 of Set 1 | cidreira
Processing Record 19 of Set 1 | bluff
Processing Record 20 of Set 1 | tasiilaq
Processing Record 21 of Set 1 | tsihombe
City not found. Skipping...
Processing Record 22 of Set 1 | os

In [ ]:
len(city_data)
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
columns = ['City', 'Country', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Current Description']
city_data_df = city_data_df[columns]
city_data_df.head()

In [5]:
city_data_df.dtypes

NameError: name 'city_data_df' is not defined

In [ ]:
# Create the output File (CSV)
#output_data_file = "WeatherPy_Database.csv"
# Export the city_data into a csv
#city_data_df.to_csv(output_data_file, index_label="City_ID")